In [1]:
#Import libraries
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import metrics, svm, naive_bayes
from PIL import Image

In [2]:
#Function to extract labels from files
def extract_labels(file_path):
    with open(file_path) as file_labels:

        file_lines = file_labels.readlines()

        file_shape_labels= [None] * len(file_lines)
        file_position_labels = [None] * len(file_lines)
        file_phoneme_labels = [None] * len(file_lines)
        file_second_phoneme_label = [None] * len(file_lines)
        
        for i, line in enumerate(file_lines):
            file_line = line.split()

            file_shape_labels[i]  = int(file_line[1])
            file_position_labels[i] = int(file_line[2])
            file_phoneme_labels[i] = int(file_line[3])
            file_second_phoneme_label[i] = int(file_line[4])
            
    return np.array(file_shape_labels), np.array(file_position_labels), np.array(file_phoneme_labels), np.array(file_second_phoneme_label)

In [3]:
#Funtion to randomly shuffle data
def shuffle_data(images, shape_labels, position_labels, phoneme_labels, second_phoneme_labels):
    #index = np.arange(len(labels))
    #np.shuffle(index)
    index = np.random.choice(np.arange(len(phoneme_labels)), replace = False, size = len(phoneme_labels))
    
    shuffled_images = images[index]
    shuffled_shape_labels = shape_labels[index]
    shuffled_position_labels = position_labels[index]
    shuffled_phoneme_labels = phoneme_labels[index]
    shuffled_second_phoneme_labels = second_phoneme_labels[index]
    return shuffled_images, shuffled_shape_labels, shuffled_position_labels, shuffled_phoneme_labels, shuffled_second_phoneme_labels

In [4]:
#Function to split data
def split_data(data, split_ratio=0.9):
    partition = round(len(data) * split_ratio)
    
    train = data[:partition]
    test = data[partition:]
    return train, test

In [5]:
#Function to prepare dataset to input network
def create_dataset(image_dir):
    dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        dataset.append(image_as_array)
    return np.array(dataset)

In [6]:
#Function to concatenate outputs of two networks into single vector
def join_outputs(out1, out2):
    joint_output = []
    for i in range(len(out1)):
        joint = np.concatenate((out1[i],out2[i]))
        joint_output.append(joint)
    return np.array(joint_output)

In [7]:
#Function to plot training and validation loss and accuracy
#Obtained from https://www.tensorflow.org/tutorials/images/classification
def plot_training_results(history):
    epochs = 10
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.show()
    return

In [8]:
#Defining the data augmentation layer that will go in at the start of the networks
#Obtained from https://www.tensorflow.org/tutorials/images/classification
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(256, 256, 3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ])

In [9]:
#Open image folders
consonants_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/ConsonantImages/')
vowels_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/VowelImages/')
combined_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/PhonemeImages/')

#Open label files
consonant_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Consonant.txt"
vowel_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Vowel.txt"
combined_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_All.txt"

In [10]:
#Extract labels
(vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used) = extract_labels(vowel_labels)
(consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this) = extract_labels(consonant_labels)
(combined_shape_labels, combined_position_labels, combined_consonant_labels, combined_vowel_labels) = extract_labels(combined_labels)

In [11]:
#Consonants Model

In [12]:
#Prepare images of consonants to input into network
consonant_images = create_dataset(consonants_dir)
consonant_images = consonant_images.reshape(len(consonant_images), 256, 256, 3)

In [13]:
#Randomly shuffle consonant images and labels
shuffled_consonants, shuffled_consonant_shape_labels, shuffled_consonant_position_labels, shuffled_consonant_phoneme_labels, nor_this = shuffle_data(consonant_images, consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this)

#Split training and test data
consonant_train, consonant_test = split_data(shuffled_consonants)
consonant_shape_labels_train, consonant_shape_labels_test = split_data(shuffled_consonant_shape_labels)
consonant_position_labels_train, consonant_position_labels_test = split_data(shuffled_consonant_position_labels)
consonant_phoneme_labels_train, consonant_phoneme_labels_test = split_data(shuffled_consonant_phoneme_labels)

In [14]:
#Build the network to recognise hand shape
hand_shape_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the network
hand_shape_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
hand_shape_model.fit(consonant_train, consonant_shape_labels_train, epochs=7)

Epoch 1/7
43/43 [==============================] - 53s 1s/step - loss: 2.2967 - accuracy: 0.2031
Epoch 2/7
43/43 [==============================] - 52s 1s/step - loss: 1.7901 - accuracy: 0.3388
Epoch 3/7
43/43 [==============================] - 53s 1s/step - loss: 1.2507 - accuracy: 0.5745
Epoch 4/7
43/43 [==============================] - 52s 1s/step - loss: 0.7815 - accuracy: 0.7413
Epoch 5/7
43/43 [==============================] - 52s 1s/step - loss: 0.5291 - accuracy: 0.8147
Epoch 6/7
43/43 [==============================] - 54s 1s/step - loss: 0.3507 - accuracy: 0.8836
Epoch 7/7
43/43 [==============================] - 56s 1s/step - loss: 0.2536 - accuracy: 0.9222


In [15]:
#plot_training_results(hand_shape_history)

In [16]:
#Test the network and print it's performance
hand_shape_test_loss, hand_shape_test_accuracy = hand_shape_model.evaluate(consonant_test, consonant_shape_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(hand_shape_test_loss, hand_shape_test_accuracy*100))

5/5 [==============================] - 2s 337ms/step - loss: 0.5808 - accuracy: 0.8400
Test loss: 0.5808157324790955, Test accuracy: 83.99999737739563


In [17]:
#Build network to recognise consonant phoneme
consonant_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
consonant_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
#consonant_phoneme_history = 
consonant_phoneme_history = consonant_phoneme_model.fit(consonant_train, consonant_phoneme_labels_train, epochs=7)

Epoch 1/7
43/43 [==============================] - 62s 1s/step - loss: 3.0167 - accuracy: 0.1023
Epoch 2/7
43/43 [==============================] - 54s 1s/step - loss: 2.7047 - accuracy: 0.2046
Epoch 3/7
43/43 [==============================] - 56s 1s/step - loss: 2.2637 - accuracy: 0.3447
Epoch 4/7
43/43 [==============================] - 58s 1s/step - loss: 1.7294 - accuracy: 0.4863
Epoch 5/7
43/43 [==============================] - 63s 1s/step - loss: 1.3494 - accuracy: 0.5678
Epoch 6/7
43/43 [==============================] - 58s 1s/step - loss: 1.0945 - accuracy: 0.6264
Epoch 7/7
43/43 [==============================] - 53s 1s/step - loss: 0.8725 - accuracy: 0.7079


In [18]:
#plot_training_results(consonant_phoneme_history)

In [19]:
#Test the network and print it's performance
consonant_phoneme_test_loss, consonant_phoneme_test_accuracy = consonant_phoneme_model.evaluate(consonant_test, consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(consonant_phoneme_test_loss, consonant_phoneme_test_accuracy*100))

5/5 [==============================] - 2s 308ms/step - loss: 1.6952 - accuracy: 0.5133
Test loss: 1.6952332258224487, Test accuracy: 51.33333206176758


In [20]:
#Obtain decisions from both networks and concatenate into a single decision vector
hand_shape_decision = hand_shape_model(consonant_train)

consonant_phoneme_decision = consonant_phoneme_model(consonant_train)

consonant_decision = join_outputs(hand_shape_decision, consonant_phoneme_decision)

In [21]:
#Obtain test vectors to test network on
hand_shape_decision_test = hand_shape_model(consonant_test)

consonant_phoneme_decision_test = consonant_phoneme_model(consonant_test)

consonant_decision_test = join_outputs(hand_shape_decision_test, consonant_phoneme_decision_test)

In [22]:
#Build network to make final decision of phoneme
consonant_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
consonant_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
consonant_decision_model.fit(consonant_decision, consonant_phoneme_labels_train, epochs=30)

Epoch 1/30
43/43 [==============================] - 0s 2ms/step - loss: 3.1427 - accuracy: 0.0563
Epoch 2/30
43/43 [==============================] - 0s 2ms/step - loss: 2.9714 - accuracy: 0.3647
Epoch 3/30
43/43 [==============================] - 0s 2ms/step - loss: 2.7593 - accuracy: 0.4996
Epoch 4/30
43/43 [==============================] - 0s 2ms/step - loss: 2.5058 - accuracy: 0.5278
Epoch 5/30
43/43 [==============================] - 0s 2ms/step - loss: 2.2280 - accuracy: 0.5434
Epoch 6/30
43/43 [==============================] - 0s 2ms/step - loss: 1.9575 - accuracy: 0.5797
Epoch 7/30
43/43 [==============================] - 0s 2ms/step - loss: 1.7158 - accuracy: 0.6086
Epoch 8/30
43/43 [==============================] - 0s 2ms/step - loss: 1.5120 - accuracy: 0.6338
Epoch 9/30
43/43 [==============================] - 0s 2ms/step - loss: 1.3496 - accuracy: 0.6590
Epoch 10/30
43/43 [==============================] - 0s 2ms/step - loss: 1.2192 - accuracy: 0.6909
Epoch 11/30
43/43 [

In [23]:
#Test network and print result
consonant_test_loss, consonant_test_accuracy = consonant_decision_model.evaluate(consonant_decision_test, consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(consonant_test_loss, consonant_test_accuracy*100))

5/5 [==============================] - 0s 2ms/step - loss: 1.4518 - accuracy: 0.5000
Test loss: 1.4517922401428223, Test accuracy: 50.0


In [24]:
#Build SVM classifier for consonant
consonant_svm = svm.SVC(kernel='linear')

consonant_svm.fit(consonant_decision, consonant_phoneme_labels_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [25]:
#Test SVM classifier and print accuracy
svm_consonant_decision = consonant_svm.predict(consonant_decision_test)

print("Accuracy:",metrics.accuracy_score(consonant_phoneme_labels_test, svm_consonant_decision)*100)

Accuracy: 55.333333333333336


In [26]:
#Build Naive-Bayes classifier for consonant
consonant_nb = naive_bayes.GaussianNB()

consonant_nb.fit(consonant_decision, consonant_phoneme_labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [27]:
#Test Naive-Bayes classifier and print accuracy
nb_consonant_decision = consonant_nb.predict(consonant_decision_test)

print("Accuracy:",metrics.accuracy_score(consonant_phoneme_labels_test, nb_consonant_decision)*100)

Accuracy: 32.666666666666664
